## Bitcoin data analytics
*This work uses only (generic) Python*

**Dataset Description:**

The dataset contains OHLC-Open, High, Low, Close (Price) daily prices for Bitcoin for about 10
years. There are 3,915 rows and 7 columns.
1. Date: month/day/year
2. Price: last (the so-called "Close") price
3. Open: opening price for the day
4. High: max price of the day
5. Low: min price for the day
6. Change%: percent change in price from previous day (the
so-called "return")

In [179]:
import csv

with open("Bitcoin_Data.csv", "r") as f:
    reader = csv.DictReader(f)
    data = [row for row in reader]

In [180]:
for row in data[:3]:
    print(row)

{'Date': '9/18/2024', 'Price': '61,757.60', 'Open': '60,308.50', 'High': '61,757.60', 'Low': '59,210.70', 'Vol.': '105.57K', 'Change %': '2.40%'}
{'Date': '9/17/2024', 'Price': '60,309.10', 'Open': '58,213.10', 'High': '61,309.00', 'Low': '57,630.20', 'Vol.': '106.30K', 'Change %': '3.60%'}
{'Date': '9/16/2024', 'Price': '58,213.10', 'Open': '59,126.20', 'High': '59,204.30', 'Low': '57,527.80', 'Vol.': '87.60K', 'Change %': '-1.56%'}


Y = 2014 + L, where L = 5

My BU ID number ends with 5, so I’ll work with bitcoin data for 2019 only.

In [181]:
from datetime import datetime

mydata =[]
y = 2014+5
for row in data:
    row['Date'] = datetime.strptime(row['Date'], "%m/%d/%Y")
    row['Price'] = float(row['Price'].replace(",", ""))
    row['High'] = float(row['High'].replace(",", ""))
    row['Low'] = float(row['Low'].replace(",", ""))
    row['Change %'] = float(row['Change %'].replace("%", ""))
    
    if(row['Date'].year == y):
        mydata.append(row)
        # print(row)

total_days = len(mydata)
print(f"Number of records for {y}: {total_days}")

Number of records for 2019: 365


In [182]:
def mean_value(values):
    cnt = 0
    total = 0
    for v in values:
        total += v
        cnt+=1
    return total/cnt

def find_min(values):
    minimum = values[0]
    for v in values:
        if(v<minimum):
            minimum = v
    return minimum

def find_max(values):
    maximum = values[0]
    for v in values:
        if(v>maximum):
            maximum = v
    return maximum

def std_dev(values):
    n=len(values)
    if n<2:
        return 0 if n==1 else None
    mu = mean_value(values)
    sm=0
    for v in values:
        sm += (v-mu)**2

    return (sm/(n-1))**0.5

In [183]:
price_col = [row['Price'] for row in mydata]
change_col = [row['Change %'] for row in mydata]

print(f"Average price = {mean_value(price_col)}\n")
print(f"Average return = {mean_value(change_col)}")
print(f"Minimum return = {find_min(change_col)}\nMaximum return = {find_max(change_col)}\nStd dev of returns = {std_dev(change_col)}")


Average price = 7371.821917808222

Average return = 0.2462465753424657
Minimum return = -13.59
Maximum return = 17.23
Std dev of returns = 3.6117620573362523


### Statistics for positive and negative days (by returns)

In [184]:
for row in mydata:
    row['T+'] = (row['Change %'] is not None and row['Change %'] >=0)

positive_days = sum(1 for row in mydata if row['T+'] is True)
negative_days = total_days - positive_days

print(f"There are {positive_days} T+ positive and {negative_days} T- negative days.")

There are 190 T+ positive and 175 T- negative days.


This confirms that the dataset has an even split, with slightly more positive days.

I computed descriptive statistics (average, minimum, maximum, and standard deviation) for both groups separately:

In [185]:
pos_return = [row['Change %'] for row in mydata if row['T+']]
neg_return = [row['Change %'] for row in mydata if row['T+'] is False]

print("Statistics for T+ positive days (by returns):")
print(f"Average = {mean_value(pos_return)}\nMinimum = {find_min(pos_return)}\nMaximum = {find_max(pos_return)}\nStd Dev = {std_dev(pos_return)}\n")

print("Statistics for T- negative days (by returns):")
print(f"Average = {mean_value(neg_return)}\nMinimum = {find_min(neg_return)}\nMaximum = {find_max(neg_return)}\nStd Dev = {std_dev(neg_return)}")

Statistics for T+ positive days (by returns):
Average = 2.515421052631579
Minimum = 0.0
Maximum = 17.23
Std Dev = 2.9373072975773247

Statistics for T- negative days (by returns):
Average = -2.2174285714285706
Minimum = -13.59
Maximum = -0.01
Std Dev = 2.4880534593172148


From the results, two groups roughly symmetrical: average values are similar but opposite in sign. Also, we see that positive days have higher maximum gains (17.23%), while negative days have deeper losses (-13.59%). Close standard deviation values give us stable volatility in both groups.

In [186]:
def three_highest(values, column_name):
    vals = values.copy()
    max_return_days = []
    for i in range(3):
        max_row = vals[0]
        for row in vals:
            if row[column_name] > max_row[column_name]:
                max_row = row
        max_return_days.append((max_row['Date'].strftime("%Y-%m-%d"), max_row[column_name]))
        vals.remove(max_row)
    return max_return_days

def three_lowest(values, column_name):
    vals = values.copy()
    min_return_days = []
    for i in range(3):
        min_row = vals[0]
        for row in vals:
            if row[column_name] < min_row[column_name]:
                min_row = row            
        min_return_days.append((min_row['Date'].strftime("%Y-%m-%d"), min_row[column_name]))
        vals.remove(min_row)
    return min_return_days

In [187]:
def quicksort(values, key=lambda x: x):
    if len(values) <= 1:
        return values
    pivot = values[len(values) // 2]
    pivot_key = key(pivot)

    left = [x for x in values if key(x) < pivot_key]
    middle = [x for x in values if key(x) == pivot_key]
    right = [x for x in values if key(x) > pivot_key]

    return quicksort(left, key) + middle + quicksort(right, key)

In [188]:
mydata_sorted = quicksort(mydata, key=lambda r: r['Date'])

In [189]:
print("Three days with highest returns: ", three_highest(mydata_sorted, 'Change %'))
print("Three days with lowest returns: ", three_lowest(mydata_sorted, 'Change %'))

Three days with highest returns:  [('2019-04-02', 17.23), ('2019-10-25', 16.65), ('2019-05-11', 12.59)]
Three days with lowest returns:  [('2019-06-27', -13.59), ('2019-07-16', -13.1), ('2019-09-24', -12.25)]


And days are not consecutive.

### Quantile Statistics

Using my custom functions, I calculated quartile statistics for positive and negative return days.

In [190]:
def calc_median(values):
    values_sorted = quicksort(values)
    n=len(values_sorted)
    if(n==0): return None
    if(n %2 == 0):
        return (values_sorted[n//2] + values_sorted[n//2-1])/2
    else:
        return values_sorted[n//2]
    
def calc_quartiles(values):
    values_sorted = quicksort(values)
    n=len(values_sorted)

    if(n % 2 == 0):
        first_half = pos_return[:len(pos_return)//2]
        sec_half = pos_return[len(pos_return)//2:]
    else:
        first_half = pos_return[:len(pos_return)//2]
        sec_half = pos_return[(len(pos_return)-1)//2:]
    Q1 = calc_median(first_half)
    Q3 = calc_median(sec_half)
    IQR = Q3 - Q1
    M = calc_median(values)
    return M, Q1, Q3, IQR

In [191]:
print("Quantile Statistics for T+ days:")
m_pos, q1_pos, q3_pos, iqr_pos = calc_quartiles(pos_return)
print(f"Median = {m_pos}\nQ1 = {q1_pos}\nQ3 = {q3_pos}\nIQR = {iqr_pos}\n")


print("Quantile Statistics for T- days:")
m_neg, q1_neg, q3_neg, iqr_neg = calc_quartiles(neg_return)
print(f"Median = {m_neg}\nQ1 = {q1_neg}\nQ3 = {q3_neg}\nIQR = {iqr_neg}")

Quantile Statistics for T+ days:
Median = 1.64
Q1 = 1.93
Q3 = 1.32
IQR = -0.6099999999999999

Quantile Statistics for T- days:
Median = -1.33
Q1 = 1.93
Q3 = 1.35
IQR = -0.5799999999999998


The quartile-based measures are not consistent with the averages from Question 2. For both groups, the median is outside the expected Q1–Q3 range, which suggests that extreme values (outliers) strongly influence the dataset.

This confirms that very large positive and negative returns are skewing the distribution.

IQR for both groups lower than standard deviation: most of the returns are clustered closely together, while large outliers create a higher spread in standard deviation.

In [192]:
def pearson_skewness(values):
    mu = mean_value(values)
    M = calc_median(values)
    sigma = std_dev(values)
    return (mu-M)/sigma


def galton_skewness(values):
    Q1, M, Q3, IQR = calc_quartiles(values)
    numerator = (Q3-M) - (M-Q1)
    return numerator/IQR

# Pearson’s skewness
print("Pearson’s skewness for T+ days:", pearson_skewness(pos_return))
print("Pearson’s skewness for T- days:", pearson_skewness(neg_return))


# Galton’s skew
print("\nGalton’s skew for T+ days:", galton_skewness(pos_return))
print("Galton’s skew for T- days:", galton_skewness(neg_return))

Pearson’s skewness for T+ days: 0.2980352288484156
Pearson’s skewness for T- days: -0.356675845571302

Galton’s skew for T+ days: 1.4754098360655739
Galton’s skew for T- days: 6.6206896551724155


Pearson’s skewness coefficient (S) for T+ days is right-skewed and S for T- days is left-skewed.

Galton’s skewness coefficient (G) for T+ days is nearly symmetric, very small value. G for T- days is large negative value, which means that Median doesn’t lie between Q1 and Q3. And from the formula, it makes Q3-M bigger than M-Q1.

Both measures agree on direction that T+ is slightly right-skewed, T- is left-skewed. From the results we see that outliers heavily influencing Galton’s skewness coefficient.

### Range

Next, I calculated Range as the difference between high and low price. Maximum range for negative days (2928.5) is slightly more than positive days (2478.2). Median values are nearly the same for both groups, meaning that most of the daily fluctuations are similar. Compared to the median mean values are higher, which can be affected by extra large values.

For the highest ranges, two days (June 26 and June 27) were consecutive. For the lowest range, dates are not consecutive.

In [193]:
for row in mydata:
    row['range'] = row['High'] - row['Low']

pos_range = [row['range'] for row in mydata if row['T+']]
neg_range = [row['range'] for row in mydata if row['T+'] is False]


print("For T+ positive return days:")
print(f"maximum = {max(pos_range)}\nmedian = {calc_median(pos_range)}\nmean = {mean_value(pos_range)}\n")


print("For T- negative return days:")
print(f"maximum = {max(neg_range)}\nmedian = {calc_median(neg_range)}\nmean = {mean_value(neg_range)}\n")


print("Three days with highest range: ", three_highest(mydata_sorted, 'range'))
print("Three days with lowest range: ", three_lowest(mydata_sorted, 'range'))

For T+ positive return days:
maximum = 2478.2
median = 263.5500000000002
mean = 370.6052631578947

For T- negative return days:
maximum = 2928.5
median = 260.7000000000007
mean = 372.3845714285715

Three days with highest range:  [('2019-06-27', 2928.5), ('2019-10-26', 2478.2), ('2019-06-26', 2182.5)]
Three days with lowest range:  [('2019-02-07', 31.5), ('2019-03-28', 33.29999999999973), ('2019-03-22', 33.899999999999636)]


### True Range

Next, I calculated True Range for each day. For the first day, TR = H-L since no previous day exists. The three days with highest TR values turned out to be the same as those identified before.

In [194]:
for i in range(len(mydata_sorted)):
    a = mydata_sorted[i]['High'] - mydata_sorted[i]['Low']
    if(i==0):
        mydata_sorted[i]['TR'] = a
    else:
        b = abs(mydata_sorted[i]['High'] - mydata_sorted[i-1]['Price'])
        c = abs(mydata_sorted[i]['Low'] - mydata_sorted[i-1]['Price'])
        mydata_sorted[i]['TR'] = max(a, b, c)


with open('Bitcoin_Data_Output.csv', mode="wt", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=list(mydata_sorted[0].keys()))
    writer.writeheader()
    writer.writerows(mydata_sorted)

In [195]:
print("Three days with highest true range value:", three_highest(mydata_sorted, 'TR'))

Three days with highest true range value: [('2019-06-27', 2928.5), ('2019-10-26', 2478.2), ('2019-06-26', 2182.5)]
